<a href="https://colab.research.google.com/github/gabrielaaguiv5/CS50-Introduction-to-Computer-Science/blob/main/notebooks/03_export_powerbi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================================================
# SmartChat Insight - 03_export_powerbi.ipynb
# Exportación y preparación de datasets finales para Power BI
# =========================================================

# 1️⃣ Importar librerías
import pandas as pd
from pathlib import Path

# 2️⃣ Rutas de entrada y salida
input_path = Path("data/outputs/")
final_path = Path("data/outputs/final/")
final_path.mkdir(parents=True, exist_ok=True)

# 3️⃣ Cargar archivos de análisis previos
clientes = pd.read_csv(input_path / "clientes.csv")
productos = pd.read_csv(input_path / "productos.csv")

print(f"✅ Clientes cargados: {len(clientes)} registros")
print(f"✅ Productos cargados: {len(productos)} registros")

clientes.head(5)

In [ ]:
# Normalizar nombres de columnas a formato uniforme para Power BI
clientes.columns = clientes.columns.str.lower().str.replace(" ", "_")
productos.columns = productos.columns.str.lower().str.replace(" ", "_")

# Asegurar tipos de datos correctos
clientes["primer_contacto"] = pd.to_datetime(clientes["primer_contacto"], errors="coerce")
clientes["ultimo_contacto"] = pd.to_datetime(clientes["ultimo_contacto"], errors="coerce")
clientes["dias_desde_ultimo"] = clientes["dias_desde_ultimo"].astype(int)


In [ ]:
# Crear un resumen con totales por estado
resumen_clientes = (
    clientes.groupby("estado")
    .agg(
        total_clientes=("user", "count"),
        promedio_mensajes=("mensajes", "mean"),
        dias_promedio_inactividad=("dias_desde_ultimo", "mean")
    )
    .reset_index()
)

# Agregar fecha de generación del reporte
from datetime import datetime
resumen_clientes["fecha_reporte"] = datetime.now().strftime("%Y-%m-%d")
resumen_clientes


In [ ]:
clientes.to_csv(final_path / "clientes_powerbi.csv", index=False, encoding="utf-8-sig")
productos.to_csv(final_path / "productos_powerbi.csv", index=False, encoding="utf-8-sig")
resumen_clientes.to_csv(final_path / "resumen_clientes.csv", index=False, encoding="utf-8-sig")

print("✅ Archivos finales generados:")
for file in final_path.glob("*.csv"):
    print("-", file.name)
